## Verbindungsaufbau

In [ ]:
from cqapi.user_editor import Conquery
eva = Conquery()
eva.login(token="conqueryToken")

In [ ]:
eva.change_dataset("fdb_demo")

In [ ]:
%matplotlib inline

In [ ]:
import seaborn as sns
from ipywidgets import interact
from matplotlib import pyplot as plt
sns.set()

import numpy as np
from matplotlib.colors import to_rgb, to_hex, to_rgba, ListedColormap

ingef_color_rgb = [
    [157, 29, 13],  # red
    [50, 79, 95],  # dark blue
    [152, 167, 175],  # blue
    [218, 218, 218],  # gray
    [204, 211, 215]  # light blue
]

yellow_in_hex = "#cabb9f"

ingef_color_rgba = [[col / 256 for col in rgb_color] + [1] for rgb_color in ingef_color_rgb]

ingefcmp = ListedColormap(colors=ingef_color_rgba, N=len(ingef_color_rgb))

ingef_color_label_to_ind = {
    "gray": 3,
    "light blue": 4,
    "blue": 2,
    "dark blue": 1,
    "red": 0
}

boxplot_saturation = 1


def get_color_by_type(colormap=ingefcmp, color_type='rgb', color=None, tint_factor=0):
    """
    Returns color or list of colors from colormap in color_type
    :param colormap: colormap from which to choose from - default: ingefcmp
    :param color_type: color_type (hex,rgba,rgb) - default: rgb
    :param color: number of color in colormap. When None, all colors in a list - default: None
    :param tint_factor: factor by which a tint of the color is created - default: 0 (no tint)
    :return: color value(s)

    Example:
    1. use color for maplotlib (hex or rgba)
       plt.fun(color=u_colors.get_color_by_type(color_type='hex', color="red"))
    """
    col_list = []

    for col_ind in range(colormap.N):
        # make tint of color
        rgb_color = to_rgb(colormap(col_ind))
        rgb_color_tint = tuple([col + (1 - col) * tint_factor for col in rgb_color])
        c = rgb_color_tint

        if color_type == 'hex':
            col_list.append(to_hex(c))
        elif color_type == 'rgb':
            col_list.append(to_rgb(c))
        elif color_type == 'rgba':
            col_list.append(to_rgba(c))
        else:
            raise ValueError(f"Unknown color type {color_type}")

    if color is None:
        return col_list

    if isinstance(color, int):
        if color not in range(colormap.N):
            raise ValueError(f"Color as integer not in {range(colormap.N)}")
    elif isinstance(color, str):
        color = ingef_color_label_to_ind.get(color, None)
        if color is None:
            raise ValueError(f"Color as string not in {ingef_color_label_to_ind.keys()}")
    else:
        raise ValueError(f"Parameter {color=} must be of type int or str or None")

    return col_list[color]


def get_quantile_colors(color: int = None, n_quantiles=4):
    """
    Takes input color and creates n_quantiles shades of it
    """
    if color is None:
        color = 1
    color_list = [get_color_by_type(color=color, tint_factor=i)
                  for i in reversed(np.arange(0, 1, 1 / n_quantiles))]

    return color_list


def get_nan_color():
    return to_rgb(yellow_in_hex)

sns.set_palette(sns.color_palette(get_color_by_type(color_type='hex')))

In [ ]:
def get_data(start_year, end_year):
    start_date=f"01.01.{start_year}"
    end_date=f"31.12.{end_year}"
    
    age_query = eva.new_query(concept_id="icd.f00-f99.f00-f09.f00", start_date=start_date, end_date=end_date)
    query = eva.new_absolute_export_query(editor_query=age_query,
                                     start_date=start_date, end_date=end_date, resolution="QUARTERS")
    eva.add_feature_to_absolute_export_form_query(query=query,
                                                         concept_id="alter")
    eva.add_feature_to_absolute_export_form_query(query=query,
                                                         concept_id="geschlecht")
    eva.add_feature_to_absolute_export_form_query(query=query,
                                                         concept_id="efn")
    eva.add_feature_to_absolute_export_form_query(query=query,
                                                         concept_id="leistungskosten_2")
    
    
    data = eva.get_data(query)
    data = data[data["Zeiteinheit"] != "Gesamt"]
    data["Zeit"] = data["Zeiteinheit"] + " " + (data["Index Zeiteinheit"].astype(int) % 4 + 1).astype(str)
    
    return data
data = get_data("2018", "2021")

In [ ]:
import numpy as np
def plotting(data, year):
    
    rows = data["Zeitraum"].apply(lambda x: x["min"].year == year)
    data = data[rows].sort_values(by="Zeit")
        
    year_rows = data["Zeiteinheit"] == "Jahr" 
    quarter_rows = data["Zeiteinheit"] == "Quartal"
    
    fig, axes = plt.subplots(2,2, figsize=(15,15))
    
    axes[0][0].set_title("Altersverteilung", fontsize=20)
    sns.histplot(ax=axes[0][0], data=data[year_rows], x="Alter - Ausgabe Alter")
    axes[0][0].set(xlabel=None, ylabel=None)
    
    axes[0][1].set_title("Geschlechterverteilung", fontsize=20)
    sns.countplot(ax=axes[0][1], data=data[year_rows], x="Geschlecht - Ausgabe Geschlecht")
    axes[0][1].set(xlabel=None, ylabel=None)
    
    axes[1][0].set_title("Krankenhausfälle", fontsize=20)
    sns.lineplot(ax=axes[1][0], data=data[quarter_rows], x="Zeit", y="Arztfälle (Gesamt) - Anzahl Arztfälle",
                estimator=np.sum)
    axes[1][0].set(xlabel=None, ylabel=None)
    #sns.countplot(ax=axes[1][0], data=data[quarter_rows & (~data["Arztfälle (Gesamt) - Anzahl Arztfälle"].isna())], x="Zeit")

    axes[1][1].set_title("Kosten", fontsize=20)                                           
    sns.lineplot(ax=axes[1][1], data=data[quarter_rows], x="Zeit", 
                 y="Leistungskosten (KH+AM+AMB+DIA+HIMI+HEMI ohne KG) - Kostensumme",
                estimator=np.sum)
    axes[1][1].set(xlabel=None, ylabel=None)
    
    
plotting(data.copy(), 2018)



In [ ]:
data

In [ ]:
@interact(Jahr=[2018, 2019, 2020, 2021])
def plot(Jahr):
    plotting(data.copy(), Jahr)
    
    

## Zugang zu bereits Gespeicherte Anfragen

In [ ]:
query = eva.from_existing_query("COVID und POST COVID oder PIMS", get_original=True)

In [ ]:
eva.get_data(query)

In [ ]:
query.show_json()

## Erstellen eigener Anfragen

In [ ]:
eva.show_concepts()

In [ ]:
eva.search_concept("icd", "covid")

In [ ]:
eva.show_concept("icd", show_all=False)

In [ ]:
eva.show_connector("icd.kh_diagnose_icd_code")

In [ ]:
query = eva.new_query(concept_id="icd.u00-u99.u00-u49.u07.u07_1", 
                      connector_ids=["icd.kh_diagnose_icd_code"],
                      select_ids=["icd.kh_diagnose_icd_code.anzahl_krankenhausfaelle",
                                  "icd.kh_diagnose_icd_code.anzahl_krankenhaustage",
                                  "icd.kh_diagnose_icd_code.liste_erster_aufnahmetag",
                                  "icd.kh_diagnose_icd_code.summe_beatmung_std"],
                      start_date="01.01.2021", end_date="31.12.2021")

In [ ]:
query.show_json()

In [ ]:
data = eva.get_data(query)

data

In [ ]:
import seaborn as sns
data["Aufnahmewoche"] = data["ICD - U07.1 - KH-Diagnose Ausgabe erster Aufnahmetag"].apply(lambda x: x.week % 52)
data["Aufnahmejahr"] = data["ICD - U07.1 - KH-Diagnose Ausgabe erster Aufnahmetag"].apply(lambda x: x.year)
data = data[data["Aufnahmejahr"] == 2021]
sns.histplot(data=data, x="Aufnahmewoche", binwidth=1)